# Chainer ローカル環境での学習

本チュートリアルでは、Azure Machine Learning service Python SDK を利用して、Chainerでのモデル学習をローカル環境で実行します。学習におけるメトリックをクラウドに記録・共有いたします。


## Python SDK バージョン確認

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

SDK version: 1.0.18


## ワークスペースへの接続
Azure Machine Learning service の [ワークスペース](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) に接続します。Azureに対する認証が必要になります。

In [2]:
from azureml.core.workspace import Workspace

ws = Workspace.get(name='azureml', 
                      subscription_id='9c0f91b8-eb2f-484c-979c-15848c098a6b', 
                      resource_group='dllab'
                   )
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: azureml
Azure region: eastus
Subscription id: 9c0f91b8-eb2f-484c-979c-15848c098a6b
Resource group: dllab


## ローカル環境でのモデル学習

### Experiment "実験" の作成
[Experiment"実験"](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) を作成し、Chainerによるモデル学習をトラックします。

In [3]:
from azureml.core import Experiment

experiment_name = 'chainer-mnist-local'
experiment = Experiment(ws, name=experiment_name)

In [4]:
run = experiment.start_logging()

### モデル学習

In [5]:
import argparse

import numpy as np

import chainer
from chainer import backends
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions
from chainer.dataset import concat_examples
from chainer.backends.cuda import to_cpu

In [6]:
class MyNetwork(Chain):

    def __init__(self, n_mid_units=100, n_out=10):
        super(MyNetwork, self).__init__()
        with self.init_scope():
            self.l1 = L.Linear(None, n_mid_units)
            self.l2 = L.Linear(n_mid_units, n_mid_units)
            self.l3 = L.Linear(n_mid_units, n_out)

    def forward(self, x):
        h = F.relu(self.l1(x))
        h = F.relu(self.l2(h))
        return self.l3(h)

In [7]:
# Download the MNIST data if you haven't downloaded it yet
train, test = datasets.mnist.get_mnist(withlabel=True, ndim=1)

In [8]:
gpu_id = -1
batchsize = 10
epochs = 5

In [9]:
run.log('Batch size', np.int(batchsize))
run.log('Epochs', np.int(epochs))

train_iter = iterators.SerialIterator(train, batchsize)
test_iter = iterators.SerialIterator(test, batchsize,
                                     repeat=False, shuffle=False)

model = MyNetwork()

if gpu_id >= 0:
    # Make a specified GPU current
    chainer.backends.cuda.get_device_from_id(0).use()
    model.to_gpu()  # Copy the model to the GPU

# Choose an optimizer algorithm
optimizer = optimizers.MomentumSGD(lr=0.01, momentum=0.9)

# Give the optimizer a reference to the model so that it
# can locate the model's parameters.
optimizer.setup(model)

while train_iter.epoch < epochs:
    # ---------- One iteration of the training loop ----------
    train_batch = train_iter.next()
    image_train, target_train = concat_examples(train_batch, gpu_id)

    # Calculate the prediction of the network
    prediction_train = model(image_train)

    # Calculate the loss with softmax_cross_entropy
    loss = F.softmax_cross_entropy(prediction_train, target_train)

    # Calculate the gradients in the network
    model.cleargrads()
    loss.backward()

    # Update all the trainable parameters
    optimizer.update()
    # --------------------- until here ---------------------

    # Check the validation accuracy of prediction after every epoch
    if train_iter.is_new_epoch:  # If this iteration is the final iteration of the current epoch

        # Display the training loss
        print('epoch:{:02d} train_loss:{:.04f} '.format(
            train_iter.epoch, float(to_cpu(loss.array))), end='')

        test_losses = []
        test_accuracies = []
        while True:
            test_batch = test_iter.next()
            image_test, target_test = concat_examples(test_batch, gpu_id)

            # Forward the test data
            prediction_test = model(image_test)

            # Calculate the loss
            loss_test = F.softmax_cross_entropy(prediction_test, target_test)
            test_losses.append(to_cpu(loss_test.array))

            # Calculate the accuracy
            accuracy = F.accuracy(prediction_test, target_test)
            accuracy.to_cpu()
            test_accuracies.append(accuracy.array)

            if test_iter.is_new_epoch:
                test_iter.epoch = 0
                test_iter.current_position = 0
                test_iter.is_new_epoch = False
                test_iter._pushed_position = None
                break

        val_accuracy = np.mean(test_accuracies)
        print('val_loss:{:.04f} val_accuracy:{:.04f}'.format(
            np.mean(test_losses), val_accuracy))

        run.log("Accuracy", np.float(val_accuracy))

epoch:01 train_loss:0.3080 val_loss:0.1532 val_accuracy:0.9513
epoch:02 train_loss:0.0141 val_loss:0.0984 val_accuracy:0.9696
epoch:03 train_loss:0.2279 val_loss:0.0857 val_accuracy:0.9736
epoch:04 train_loss:0.0756 val_loss:0.0790 val_accuracy:0.9769
epoch:05 train_loss:0.0004 val_loss:0.0961 val_accuracy:0.9733


In [10]:
run.complete()

In [11]:
# to get more details of your run
print(run.get_details())

{'runId': 'e544c1cb-bda4-4d45-8637-5a5917f00542', 'target': 'sdk', 'status': 'Completed', 'startTimeUtc': '2019-03-20T05:55:01.159082Z', 'endTimeUtc': '2019-03-20T05:56:21.502461Z', 'properties': {'ContentSnapshotId': '1f3f1096-20b7-4d97-a00c-cf2bc5ac7890'}, 'logFiles': {}}


## 結果確認

### Azure Portal でメトリックの確認

In [12]:
run

Experiment,Id,Type,Status,Details Page,Docs Page
chainer-mnist-local,e544c1cb-bda4-4d45-8637-5a5917f00542,,Completed,Link to Azure Portal,Link to Documentation



※画面イメージ
<br>
<img src = "../images/chainer_local_portal.png" width = 1000>